In [ ]:
import sagemaker
import tensorflow as tf
import os
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()

QUESTION = 1
INSTANCE_TYPE = "ml.m4.xlarge"

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='data', key_prefix='data')

In [ ]:
def _get_n_words():
    vocab_file = os.path.join("data", "vocab_question_{}".format(QUESTION))
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocab_file)
    return len(vocab_processor.vocabulary_)



classifier = TensorFlow(entry_point='sm_transcript_classifier_ep.py',
                               name="grader_q{}".format(QUESTION),
                               role=role,
                               training_steps= 1e4,                                  
                               evaluation_steps= 100,
                               train_instance_count=1,
                               train_instance_type=INSTANCE_TYPE,
                               py_version="py2",
                               framework_version="1.5",
                               hyperparameters={
                                   "question": QUESTION,
                                   "n_words": _get_n_words()
                               })

classifier.fit(inputs)

In [ ]:
endpoint_name = "sagemaker-tensorflow-py2-cpu-2018-04-11-15-40-39-797"
classifier = classifier.deploy(initial_instance_count=1, instance_type=INSTANCE_TYPE, endpoint_name=endpoint_name)